In [0]:
from pyspark.sql.functions import *

# Employee DF
schema = 'EMPLOYEE_NUMBER int, MANAGER_EMPLOYEE_NUMBER int'
employees = spark.createDataFrame(
[[801,None], 
[1016,801], 
[1003,801], 
[1019,801], 
[1010,1003], 
[1004,1003], 
[1001,1003],
[1012,1004], 
[1002,1004], 
[1015,1004], 
[1008,1019], 
[1006,1019], 
[1014,1019],
[1011,1019]], schema=schema)

# initial DataFrame
empDF = employees \
    .withColumnRenamed('EMPLOYEE_NUMBER', 'level_0') \
    .withColumnRenamed('MANAGER_EMPLOYEE_NUMBER', 'level_1')

i = 1

# Loop Through if you dont know recusrsive depth
while True:
    this_level = 'level_{}'.format(i)
    next_level = 'level_{}'.format(i+1)

    """
        print(this_level, next_level)
        level_1 level_2
        level_2 level_3
        level_3 level_4
    """

    emp_level = employees \
        .withColumnRenamed('EMPLOYEE_NUMBER', this_level) \
        .withColumnRenamed('MANAGER_EMPLOYEE_NUMBER', next_level)

    empDF = empDF.join(emp_level, on=this_level, how='left')
  
    # Check if DF is empty. Break loop if empty, Otherwise continue with next level
    if empDF.where(col(next_level).isNotNull()).count() == 0:
        break
    else:
        i += 1

# Sort columns and show
empDF.select('level_0', 'level_1', 'level_2', 'level_3').show()

+-------+-------+-------+-------+
|level_0|level_1|level_2|level_3|
+-------+-------+-------+-------+
|    801|   null|   null|   null|
|   1016|    801|   null|   null|
|   1003|    801|   null|   null|
|   1019|    801|   null|   null|
|   1010|   1003|    801|   null|
|   1004|   1003|    801|   null|
|   1001|   1003|    801|   null|
|   1012|   1004|   1003|    801|
|   1002|   1004|   1003|    801|
|   1015|   1004|   1003|    801|
|   1008|   1019|    801|   null|
|   1006|   1019|    801|   null|
|   1014|   1019|    801|   null|
|   1011|   1019|    801|   null|
+-------+-------+-------+-------+



In [0]:
subsidiary = spark.createDataFrame(
    [
        ["C1", "C2"],
        ["C2", "C3"],
        ["C2", "C5"],
        ["C4", "C6"]
    ], ["parent", "child"]
)

subsidiary.show()

subsidiary_df = subsidiary.withColumnRenamed("child", "level_0").withColumnRenamed("parent", "level_1")
subsidiary_df.show()

+------+-----+
|parent|child|
+------+-----+
|    C1|   C2|
|    C2|   C3|
|    C2|   C5|
|    C4|   C6|
+------+-----+

+-------+-------+
|level_1|level_0|
+-------+-------+
|     C1|     C2|
|     C2|     C3|
|     C2|     C5|
|     C4|     C6|
+-------+-------+



In [0]:
"""
https://www.youtube.com/watch?v=zxcLgJkR_LI

-- Investments
INSERT INTO Invested (pname, cname) VALUES
('Don', 'C1'),
('Don', 'C4'),
('Ron', 'C1'),
('Hil', 'C3');

-- Subsidiary relationships
INSERT INTO Subsidiary (parent, child) VALUES
('C1', 'C2'),
('C2', 'C3'),
('C2', 'C5'),
('C4', 'C6');

select * from Invested ;
select * from Subsidiary ;

SQL SOlution but Recursive cte is not supported in Spark
with recursive cte as (
  select parent, child from Subsidiary
  union all
  select cte.parent, s.child
  from cte inner join Subsidiary s 
  on cte.child = s.parent
)
select p.pname, c.pname from cte 
inner join Invested p 
on cte.parent = p.cname
inner join Invested c
on cte.child = c.cname ;
 

"""
from pyspark.sql.functions import *

# Employee DF
schema = 'EMPLOYEE_NUMBER string, MANAGER_EMPLOYEE_NUMBER string'
employees = spark.createDataFrame(
    [
        ["C1", "C2"],
        ["C2", "C3"],
        ["C2", "C5"],
        ["C4", "C6"]
    ], schema=schema)

# initial DataFrame
empDF = employees \
    .withColumnRenamed('EMPLOYEE_NUMBER', 'level_0') \
    .withColumnRenamed('MANAGER_EMPLOYEE_NUMBER', 'level_1')

i = 1

# Loop Through if you dont know recusrsive depth
while True:
    this_level = 'level_{}'.format(i)
    next_level = 'level_{}'.format(i+1)

    emp_level = employees \
        .withColumnRenamed('EMPLOYEE_NUMBER', this_level) \
        .withColumnRenamed('MANAGER_EMPLOYEE_NUMBER', next_level)

    empDF = empDF.join(emp_level, on=this_level, how='left')
  
    # Check if DF is empty. Break loop if empty, Otherwise continue with next level
    if empDF.where(col(next_level).isNotNull()).count() == 0:
        break
    else:
        i += 1

# Sort columns and show
empDF.select('level_0', 'level_1', 'level_2').show()


+-------+-------+-------+
|level_0|level_1|level_2|
+-------+-------+-------+
|     C1|     C2|     C5|
|     C1|     C2|     C3|
|     C2|     C3|   null|
|     C2|     C5|   null|
|     C4|     C6|   null|
+-------+-------+-------+



In [0]:
empDF.createOrReplaceTempView("emp")

subsidiary_df = spark.sql("""
          with cte as (
            select level_0, level_1 from emp
            union
            select level_0, level_2 from emp
            )
            select * from cte where level_1 is not null
          """)
          
subsidiary_df.show()

+-------+-------+
|level_0|level_1|
+-------+-------+
|     C1|     C2|
|     C2|     C3|
|     C2|     C5|
|     C4|     C6|
|     C1|     C5|
|     C1|     C3|
+-------+-------+



In [0]:
investment_df = spark.createDataFrame(
    [
        ['Don', 'C1'],
        ['Don', 'C4'],
        ['Ron', 'C1'],
        ['Hil', 'C3']
    ],
    ["pname", "cname"]
)

investment_df.show()

+-----+-----+
|pname|cname|
+-----+-----+
|  Don|   C1|
|  Don|   C4|
|  Ron|   C1|
|  Hil|   C3|
+-----+-----+



In [0]:
df1 = subsidiary_df.join(investment_df, subsidiary_df["level_0"] == investment_df["cname"], "inner")\
    .withColumnRenamed("level_1", "level_1_i").withColumnRenamed("pname", "pname_i").withColumnRenamed("cname", "cname_i")

df1.show()

+-------+-------+-------+-------+
|level_0|level_1|pname_i|cname_i|
+-------+-------+-------+-------+
|     C1|     C3|    Don|     C1|
|     C1|     C5|    Don|     C1|
|     C1|     C2|    Don|     C1|
|     C4|     C6|    Don|     C4|
|     C1|     C3|    Ron|     C1|
|     C1|     C5|    Ron|     C1|
|     C1|     C2|    Ron|     C1|
+-------+-------+-------+-------+



In [0]:
subsidiary_df.createOrReplaceTempView("cte")
investment_df.createOrReplaceTempView("Invested")

spark.sql("""
            select c.pname as cname, p.pname as pname 
            from cte 
            inner join Invested p 
            on cte.level_1 = p.cname
            inner join Invested c
            on cte.level_0 = c.cname
          """).show()

+-----+-----+
|cname|pname|
+-----+-----+
|  Don|  Hil|
|  Ron|  Hil|
+-----+-----+

